In [ ]:
import shapely
from psql_utils.epsql import Engine
from psql_utils.pgis_utils import find_nhgis_census_table, get_geom_at_iloc
import geopandas as gpd
import pandas as pd

engine = Engine()

In [ ]:
# Get Oakland and its neighborhoods
neighborhoods = engine.execute_returning_gdf("select hood, geom from allegheny_county_neighborhoods where hood like '%%Oakland%%'")
oakland_geom = neighborhoods.geometry.unary_union
neighborhoods = gpd.GeoDataFrame( # type: ignore    
    pd.concat([neighborhoods, gpd.GeoDataFrame({'hood': 'Oakland', 'geom': oakland_geom}, index=[0])], ignore_index=True),
    geometry='geom',
    crs=neighborhoods.crs)
print(neighborhoods)
print(neighborhoods.crs.to_epsg())
#neighborhoods.explore()

In [ ]:

        
#    return engine.execute_returning_gdf(f"select *, st_area(st_intersection(geom, {pggeom(geom)})::geography) / st_area(geom::geography) as weight from {geo_table} where st_intersects({pggeom(geom)}, geom)")

#    print(table)

In [ ]:
# Fetch block-level population data from decennial census using api.census.gov
import requests
import pandas as pd

# Feel free to Mickey's key or replace with your own.
CensusAPIKey = open(".secrets/census_api_key.txt").read().strip()

# Define API endpoint
#base_url = 'https://api.census.gov/data/2021/acs/acs5'
base_url = 'https://api.census.gov/data/2020/dec/pl'

measures = {
    'P1_001N': 'Total Population',
    'P2_002N': 'Hispanic/Latino',
    'P2_005N': 'White',
    'P2_006N': 'Black or African American',
    'P2_007N': 'American Indian/Alaska Native',
    'P2_008N': 'Asian',
    'P2_009N': 'Native Hawaiian/Other Pacific Islander',
    'P2_010N': 'Other',
    'P2_011N': 'Two or More Races'
}

# Define payload
payload = {
    'get': ','.join(['GEO_ID'] + list(measures.keys())),
    'for': 'block:*',
    'in': 'state:42 county:003 tract:*', # Note: hardcoding in specific states
    'key': CensusAPIKey
}

# Send request
response = requests.get(base_url, params=payload)

# Convert response to pandas dataframe
orig = pd.DataFrame(response.json()[1:], columns=response.json()[0])


In [ ]:
copy = orig.copy()
copy['GEO_ID'] = copy['GEO_ID'].str[9:]

for id, name in measures.items():
    copy[name] = copy[id].astype(int)
    del copy[id]

# Rename Columns
#ACSdf.columns = ['GEO_ID','Total Pop', 'state','county','tract','bg']

# Print dataframe

copy['Other'] = copy['American Indian/Alaska Native'] + copy['Native Hawaiian/Other Pacific Islander'] + copy['Other']
del copy['American Indian/Alaska Native']
del copy['Native Hawaiian/Other Pacific Islander']

copy

In [ ]:
from psql_utils.pgis_utils import intersect_all


for year in [2020]:
    geom_table = find_nhgis_census_table(engine, 'block', year)
    for i in range(len(neighborhoods)):
        geom = get_geom_at_iloc(neighborhoods, i)
        print(f"Year {year}, neighborhood {neighborhoods['hood'][i]}")
        df = intersect_all(engine, geom, geom_table)
        all_len = len(df)
        df = df[df.weight > 0.5]
        print(f"  {len(df)} of {all_len} blocks have > 50% overlap")
        neighborhood_geoids = set(df['geoid'])
        #print(df.sum())
        in_neighborhood = copy['GEO_ID'].apply(lambda geoid: geoid in neighborhood_geoids)
        stats = copy[in_neighborhood].drop(['GEO_ID', 'state', 'county', 'tract', 'block'], axis=1)
        print(f"  {len(stats)} blocks found from stats")
        sum = stats.sum()
        sum['hood'] = neighborhoods['hood'][i]
        sum['year'] = year
        print(sum)
        # for geoid in df.geoid:
        #     count += 
        #     copy.loc[copy['GEO_ID'] == geoid, 'hood'] = neighborhoods['hood'][i]

